<a href="https://colab.research.google.com/github/m4vic/ML-DL-projects-/blob/main/XGBclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
persona_data = pd.read_csv('/content/personality_dataset (1).csv')

In [ ]:

type(persona_data) # checking the type of the data
persona_data.head() # printing the top 5 values for dataframe


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [ ]:

persona_data.shape

(2900, 8)

In [ ]:

persona_data.describe()

,Time_spent_Alone,Social_event_attendance,Going_outside,Friends_circle_size,Post_frequency
count,2837.000000,2838.000000,2834.000000,2823.000000,2835.000000
mean,4.505816,3.963354,3.000000,6.268863,3.564727
std,3.479192,2.903827,2.247327,4.289693,2.926582
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,3.000000,1.000000
50%,4.000000,3.000000,3.000000,5.000000,3.000000
75%,8.000000,6.000000,5.000000,10.000000,6.000000
max,11.000000,10.000000,7.000000,15.000000,10.000000


In [ ]:

persona_data.isnull().sum()

,0
Time_spent_Alone,63
Stage_fear,73
Social_event_attendance,62
Going_outside,66
Drained_after_socializing,52
Friends_circle_size,77
Post_frequency,65
Personality,0


In [ ]:

persona_data = persona_data.dropna()

In [ ]:
persona_data.isnull().sum()

,0
Time_spent_Alone,0
Stage_fear,0
Social_event_attendance,0
Going_outside,0
Drained_after_socializing,0
Friends_circle_size,0
Post_frequency,0
Personality,0


In [ ]:
persona_data.replace({'Personality':{'Extrovert':0,'Introvert':1},'Stage_fear':{'No':0,'Yes':1},'Drained_after_socializing':{'No':0,'Yes':1}},inplace = True)



/tmp/ipython-input-1821062593.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  persona_data.replace({'Personality':{'Extrovert':0,'Introvert':1},'Stage_fear':{'No':0,'Yes':1},'Drained_after_socializing':{'No':0,'Yes':1}},inplace = True)


In [ ]:

x = persona_data.drop(columns = 'Personality', axis = 1) # devieding the features and labels x = all the features y = lables persnoality
y = persona_data['Personality']

In [ ]:
persona_data.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,0,4.0,6.0,0,13.0,5.0,0
1,9.0,1,0.0,0.0,1,0.0,3.0,1
2,9.0,1,1.0,2.0,1,5.0,2.0,1
3,0.0,0,6.0,7.0,0,14.0,8.0,0
4,3.0,0,9.0,4.0,0,8.0,5.0,0


In [ ]:
print(x)

      Time_spent_Alone  Stage_fear  Social_event_attendance  Going_outside  \
0                  4.0           0                      4.0            6.0   
1                  9.0           1                      0.0            0.0   
2                  9.0           1                      1.0            2.0   
3                  0.0           0                      6.0            7.0   
4                  3.0           0                      9.0            4.0   
...                ...         ...                      ...            ...   
2892               9.0           1                      2.0            0.0   
2895               3.0           0                      7.0            6.0   
2896               3.0           0                      8.0            3.0   
2897               4.0           1                      1.0            1.0   
2899               3.0           0                      6.0            6.0   

      Drained_after_socializing  Friends_circle_size  Post_freq

In [ ]:
print(y)

0       0
1       1
2       1
3       0
4       0
       ..
2892    1
2895    0
2896    0
2897    1
2899    0
Name: Personality, Length: 2477, dtype: int64


# *XGboost classifier*

In [ ]:

scaler = StandardScaler()

In [ ]:

scaler.fit(x)

StandardScaler()

In [ ]:

standardized_data = scaler.transform(x)

In [ ]:

x = standardized_data
y = persona_data['Personality']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2, stratify= y, random_state=2)


Optuna objective

In [ ]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = XGBClassifier(**params)
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1)
    return score.mean()


In [ ]:
# optuna tuning

import optuna


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-15 18:35:31,038] A new study created in memory with name: no-name-c7975baf-bb17-4575-b0b7-9146f41fd4e9
[I 2025-08-15 18:35:33,529] Trial 0 finished with value: 0.9298693778543982 and parameters: {'n_estimators': 214, 'max_depth': 8, 'min_child_weight': 9, 'learning_rate': 0.1402165655379452, 'subsample': 0.628689171686006, 'colsample_bytree': 0.7434687352956972, 'gamma': 4.031341307232486, 'reg_alpha': 2.5000165202653872, 'reg_lambda': 3.9555934968941915}. Best is trial 0 with value: 0.9298693778543982.
[I 2025-08-15 18:35:33,651] Trial 1 finished with value: 0.9415067053917019 and parameters: {'n_estimators': 313, 'max_depth': 7, 'min_child_weight': 7, 'learning_rate': 0.13514556351643855, 'subsample': 0.7499523079310753, 'colsample_bytree': 0.9174520725578533, 'gamma': 3.1692340248054314, 'reg_alpha': 2.2664023771761936, 'reg_lambda': 0.29662988700345394}. Best is trial 1 with value: 0.9415067053917019.
[I 2025-08-15 18:35:33,814] Trial 2 finished with value: 0.95107542146

In [ ]:
study.best_params

{'n_estimators': 321,
 'max_depth': 5,
 'min_child_weight': 4,
 'learning_rate': 0.23504970673218653,
 'subsample': 0.9341136850676154,
 'colsample_bytree': 0.733379661111,
 'gamma': 1.9091038838482612,
 'reg_alpha': 0.06562170036989937,
 'reg_lambda': 2.874779027167337}

In [ ]:
study.best_value

0.9539017658747372

In [ ]:
best_params = study.best_params
best_params.update({'objective': 'binary:logistic', # adding some fixed params
                    'eval_metric': 'logloss',
                    'use_label_encoder': False})

In [ ]:
best_model = XGBClassifier(**best_params) # unpacking the dixt best_params into arguments

In [ ]:
best_model.fit(x_train,y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [18:39:37] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.733379661111, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=1.9091038838482612, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.23504970673218653,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=321, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# evaluation

x_train_prediction = best_model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print(training_data_accuracy)

0.9252902574457345


In [ ]:
x_test_prediction = best_model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)

In [ ]:
print(test_data_accuracy)

0.9254032258064516
